In [1]:
%run training_functions.ipynb

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-09 19:51:13.335746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744228273.349377 3029831 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744228273.353559 3029831 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744228273.365688 3029831 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 

['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, peft, pickle, pytest, ipytest, pd, transformers, torch, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, search_data, search_with_strings, map_data, print_trainable_parameters, metric, comp_metrics_output, compute_metrics, make_trainer, get_dataframe, get_training_output, 

## Dataset value example 
{'record': 'NZ_LIXJ01000001.1', 'text': 'CGACTCCGCACAGGCCCTAGGCGTGGTCCACGCATCGAGCGCGTGCGCCGTGGCATACACCCGGGCCACCCGTACGACCCTTCCGGCCGCCCCGTGTGCGCCCTCGTCGCCGAGTCCGCTCCGGCGCGCGCGGGCGCACTGCTGTTCCGCGCGGTGTCATGCGCTCCTCCCGTCCCGACATGCGCCTTTTTGCAGGGGAGAGGGAGGGGTAGGGTCGCGCGTACGCCTAGCACGGGGGAACGGTCTTGGGGGACGCAGTGGACGTACGGGTGGCGGCGATCGCCAGTCTCACACCGCTCGAAGAACTCGACAGCGATCCGTTCCTCGTGGACACCCGCAGCCAGCACGACATGTGCGCACGCTGGGCCGCGGACAAGGGGTACGTCGTCACCCGCCAGCTCCGCTTCTACGGGCTGCGCCCCGATCACCACGCCCTGTGGAGCGATGTCGAGAGCGGCGACGTCGAGGTGTTCGTCGCCGCCAACGACCGGGTGCTGGAACGGGCCCTCACGTCGGTCGCGGAGTTCGCCGCGGAGTGCGCGCGGCGCGGCGTGCGTCTGGAGATCGCCGGGCTGGACGAGCCGCCGTACAACGCCGGCACGAAGGCGAGCGTGCATCGCAGGCTCTCCATGCCCACCGCCGGTTACGACGGCCGCTGACGCACTCCGCCTCCACTGCCGGCTCGCGGCCCCTGCCCTCGTCCCGGCCCCGTCCCTTCCGCGGCTTGCCGTCGGGCTGTGACAGGCTGGGGGACAGGGTCCGCAGCGGCGGGGCCCGGACGTGAGGTGGTTGAGGCGTGGGTGACGGGCGATGGCGAACGGTCGGCAACGCCCTGATGCGAGTGGCCGTGGTGTGGGCGGTCTCCACGCTCACGATGCTGGTGCTCGCCGGGGTGCTGCCGGAGTTCCGGCTCCAGTCGGACGACGGCGACAGCATCACCAAGACCGCGTTCACCGCGGCCTGGGGCGCCGGAGCGTTCGGCCTGCTCTCCGCACTCGTC'}

In [2]:
# let's try prompting the fine-tuned models
def continued_convo(model, tokenizer, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", device_map="auto", max_new_tokens=512, model=model, tokenizer=tokenizer)
    if convo==None:
        messages=[{"role": "system", "content": "You are a helpful assistant"}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    #tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    answer=pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
    print(answer)
    messages.append(answer)
    return pipe, messages 

In [3]:
lla_model=AutoModelForCausalLM.from_pretrained('./fine_tuned/lla2/') 

In [4]:
lla_token=AutoTokenizer.from_pretrained('fine_tuned/lla2_token/')

In [7]:
model, answer=continued_convo(lla_model, lla_token, 
                              "What are genome record numbers used for?")
model, answer2=continued_convo(lla_model, lla_token, 
                               "What would a genome record number look like?", answer)
model, answer3=continued_convo(lla_model, lla_token, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4=continued_convo(lla_model, lla_token, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000010.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': 'Genome record numbers, also known as GenBank accession numbers, are unique identifiers assigned to each genome sequence deposited in the GenBank database. These numbers serve several purposes:\n\n1. **Identification**: Each genome sequence is assigned a unique accession number, allowing researchers to easily identify and reference specific sequences.\n2. **Access**: The GenBank database provides a centralized location for storing and sharing genome sequences, making it easier for researchers to access and utilize these sequences.\n3. **Identification of new sequences**: The accession number helps to distinguish between different versions of the same sequence, which can be important for tracking changes in the sequence over time.\n4. **Verification**: The accession number can be used to verify the authenticity and integrity of a genome sequence, ensuring that it is not a duplicate or a modified version of an existing sequence.\n5. **Research and analysi

Device set to use cuda:0


{'role': 'assistant', 'content': 'A genome record number, also known as a GenBank accession number, typically consists of a combination of letters and numbers that uniquely identify a specific genome sequence. Here\'s a breakdown of what a typical genome record number looks like:\n\n**Format:** XX.XX.XX.XX\n\n**Components:**\n\n1. **Prefix:** A series of letters that identifies the organism or species (e.g., "GR" for human, "S" for salmon, etc.).\n2. **Sequence type:** A code that indicates the type of sequence (e.g., "S" for a short-read sequence, "I" for an insert sequence, etc.).\n3. **Version:** A number that indicates the version of the sequence (e.g., "1" for the first version, "2" for the second version, etc.).\n4. **Date:** A date that indicates when the sequence was deposited (e.g., "2020-01-01" for a sequence deposited in January 2020).\n5. **Number:** A series of digits that uniquely identify the sequence (e.g., "12345678" for a specific sequence).\n\n**Example:** GR23456789

Device set to use cuda:0


{'role': 'assistant', 'content': 'To identify the genome, we need to look up the record number "NZ_LIXJ01000001.1" in the GenBank database.\n\nAfter searching the database, I found that the record number "NZ_LIXJ01000001.1" corresponds to the "New Zealand domestic cat genome" (Felis catus domesticus).\n\nHere\'s a breakdown of the record number:\n\n* "NZ_LIXJ" is the prefix, indicating that the sequence is for a New Zealand domestic cat.\n* ".01000001.1" is the sequence type, indicating that the sequence is a short-read sequence.\n\nThe full name of the genome is "New Zealand domestic cat genome (Felis catus domesticus)".\n\nThis genome sequence was generated by the New Zealand National Animal Health Board (NAHB) and is part of the New Zealand Domestic Cat Genome Project. The sequence was deposited in the GenBank database in 2010.\n\nPlease note that the availability and accuracy of genome sequences can change over time, so it\'s always a good idea to verify the information with the or

In [8]:
model, answer=continued_convo(lla_model, lla_token, "Please provide a list with ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': "Here are ten example base pairs for a genome:\n\n1. A (Adenine)\n2. T (Thymine)\n3. C (Cytosine)\n4. G (Guanine)\n5. A (Adenine)\n6. T (Thymine)\n7. C (Cytosine)\n8. G (Guanine)\n9. A (Adenine)\n10. T (Thymine)\n\nThese base pairs are the fundamental building blocks of DNA and are arranged in a specific order to form the genetic code. The sequence of these base pairs determines the genetic information encoded in an organism's genome."}


In [9]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'Please provide an example of what the first \n    20 base pairs of a genome may look like'},
  {'role': 'assistant',
   'content': 'Here are twenty example base pairs for a genome: CGACGTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAGCTGTAATCGGGC

In [10]:
search_with_strings(ds_gst1_train, 'CGACGTCAAGCTAGATCGATCGATCGATCGATCGATCGATCGAT')

('CGACGTCAAGCT', 12)

In [11]:
search_with_strings(ds_gst1_train, 'CGACGTGTAATCGGGCCGTAAGCTGTAATCGGGCCGTAAG')

('CGACGTGTAA', 10)

In [13]:
model, answer=continued_convo(lla_model, lla_token, """Please provide five examples for what
    the first 20 base pairs of a genome may look like""")

Device set to use cuda:0


{'role': 'assistant', 'content': "Here are five examples of what the first 20 base pairs of a genome may look like:\n\n1. **ATCGATCGATCGATCG**: This sequence is a simple example of a palindrome, where the sequence reads the same backward as forward. It's a common sequence found in many types of DNA, including bacterial and viral genomes.\n\n2. **ATCGATCGATCGATCGATCG**: This sequence is a palindrome with a slight variation, where the first and last nucleotides are the same, and the middle nucleotides are different.\n\n3. **ATCGATCGATCGATCGATCGATCGATCGATCG**: This sequence is a more complex example of a palindrome, where the sequence is repeated multiple times, with some variations in the middle.\n\n4. **ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG**: This sequence is a palindrome with a repeating pattern of nucleotides, where the sequence is identical to itself.\n\n5. **ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG**: This sequence is a palindrome with a simple re

In [14]:
search_with_strings(ds_gst1_train, 'ATCGATCGATCGATCGATCG')

('ATCGATCGATCG', 12)

In [15]:
pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """What are the first fifty base pairs of the sequence 
    of the gene NZ_LIXJ01000001.1?"""},
    {"role": "assistant", "content": """The first fifty base pairs of gene NZ_LIXJ01000001.1 are: CGACTC"""}
]
output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What are the first fifty base pairs of the sequence \n    of the gene NZ_LIXJ01000001.1?'},
  {'role': 'assistant',
   'content': 'The first fifty base pairs of gene NZ_LIXJ01000001.1 are: CGACTCAGTATTTAAATTGCTTCTGGTGTAATCTGATCCTTCTCAGGTTTCTCTCTGGTGTAATCTGATCGGTCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTTCTCTGCTT

In [16]:
search_with_strings(ds_gst1_train, 'CGACTCAGTATTTAAATTGCTTCTGGTGTAATCTGATCCTTCTCAGG')

('CGACTCAGTATT', 12)

In [17]:
# the model above is fine-tuned using steps as the eval_strategy

In [18]:
lla_epoch=AutoModelForCausalLM.from_pretrained('./fine_tuned/lla/') 
lla_token_epoch=AutoTokenizer.from_pretrained('./fine_tuned/lla_token')

In [20]:
model, answer=continued_convo(lla_epoch, lla_token_epoch, 
                              "What are genome record numbers used for?")
model, answer2=continued_convo(lla_epoch, lla_token_epoch, 
                               "What would a genome record number look like?", answer)
model, answer3=continued_convo(lla_epoch, lla_token_epoch, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4=continued_convo(lla_epoch, lla_token_epoch, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000001.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': "Genome record numbers, also known as GenBank accession numbers, are unique identifiers assigned to each DNA sequence or genome assembly in the GenBank database. These numbers serve several purposes:\n\n1. **Identification**: Each genome sequence is assigned a unique accession number, making it easy to identify and distinguish between different DNA sequences.\n2. **Tracking**: Genome record numbers allow researchers to track the evolution and diversification of different organisms, including their phylogenetic relationships.\n3. **Reference**: These numbers serve as a reference point for comparing and analyzing different DNA sequences, helping to resolve conflicts and identify conserved regions.\n4. **Searching**: Genome record numbers enable researchers to search for specific DNA sequences or assemblies within the GenBank database.\n5. **Collaboration**: Accession numbers facilitate collaboration among researchers, as they can easily identify and acces

Device set to use cuda:0


{'role': 'assistant', 'content': "A genome record number is a unique identifier assigned to each DNA sequence or genome assembly in the GenBank database. Here's a breakdown of what a genome record number typically looks like:\n\n**Format:**\n\n* The first part of the number is a string of letters and numbers, usually 1-4 characters long, that identifies the organism or genome assembly.\n* The next part of the number is a string of letters, numbers, and special characters, usually 1-5 characters long, that represents the DNA sequence itself.\n\n**Example:**\n\n* Organism accession number: NC_000001\n* DNA sequence: ATCG:ATGC\n* Example of a full genome record number: NC_000001.11.1\n\n**Breakdown:**\n\n* The first part of the number (organism accession number) is usually a string of letters and numbers that identifies the organism or genome assembly, such as:\n\t+ Organism name (e.g., Homo sapiens, Escherichia coli)\n\t+ Organism accession number (e.g., NC_000001, ATCG:ATGC)\n\t+ Organi

Device set to use cuda:0


{'role': 'assistant', 'content': 'To identify the genome, we need to look at the record number "NZ_LIXJ01000001.1".\n\nThe record number "NZ_LIXJ01000001.1" identifies the following:\n\n* Organism: New Zealand flagtail (Lixus spp.)\n* Genome assembly: New Zealand flagtail genome (NZ_LIXJ01000001)\n* Genome assembly name: New Zealand flagtail genome assembly (NZ_LIXJ01000001)\n\nSo, the genome identified by this record number is the New Zealand flagtail genome, which is a specific assembly of the New Zealand flagtail (Lixus spp.) genome.\n\nHere\'s a breakdown of the record number:\n\n* "NZ_LIXJ" is likely the organism identifier, which identifies the species (New Zealand flagtail) and the genus (Lixus).\n* "01000001" is likely the genome assembly identifier, which identifies the specific genome assembly (NZ_LIXJ01000001).\n* ".1" is likely the genome assembly name, which identifies the specific genome assembly (New Zealand flagtail genome assembly NZ_LIXJ01000001).\n\nSo, the genome id

In [ ]:
# the model above is fine-tuned using epochs as the eval_strategy